In [2]:
## Importing the relevant packages:

import pandas as pd
import numpy as np
import requests
import io
import sys
import os
from ast import literal_eval

import matplotlib.pyplot as plt

from scipy.stats.mstats import winsorize

In [3]:
survey_df = pd.read_csv("decision_maker_survey_data_with_titles.csv")
print(len(survey_df))
survey_df.head()

442


,Unnamed: 0.1,Unnamed: 0,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,...,Unnamed: 165,Unnamed: 166,role_type.1,role_type_11_TEXT.1,final_role_type,final_DM,DM,over_claimed_title,final_marketing_role,self_reported_marketing_role
0,0,0,12/7/2022 15:09,12/7/2022 15:14,IP Address,192.55.55.56,100,335,True,12/7/2022 15:14,...,NaN,NaN,Business and finance specialist (business spe...,NaN,Business and finance specialist (business spe...,0,0,NaN,0,0
1,1,1,12/7/2022 15:08,12/7/2022 15:15,IP Address,24.6.9.216,100,430,True,12/7/2022 15:15,...,NaN,NaN,"Top executive (e.g. chief executives, general...",NaN,"Top executive (e.g. chief executives, general...",1,1,NaN,0,0
2,2,2,12/7/2022 15:07,12/7/2022 15:16,IP Address,141.239.183.125,100,481,True,12/7/2022 15:16,...,NaN,NaN,Operations manager (e.g. managers of administ...,NaN,Operations manager (e.g. managers of administ...,1,1,NaN,0,0
3,3,3,12/7/2022 15:14,12/7/2022 15:19,IP Address,111.69.48.150,100,334,True,12/7/2022 15:19,...,NaN,NaN,"Top executive (e.g. chief executives, general...",NaN,"Top executive (e.g. chief executives, general...",1,1,NaN,0,0
4,4,4,12/7/2022 15:13,12/7/2022 15:24,IP Address,73.241.86.146,100,653,True,12/7/2022 15:24,...,NaN,NaN,"Manager of advertising, marketing, promotions,...",NaN,"Manager of advertising, marketing, promotions,...",1,1,NaN,1,1


In [4]:
survey_df['final_role_type'].value_counts()

 Top executive (e.g. chief executives, general and operations managers of multiple departments or locations, legislators)                                                                                                            248
Other manager  (managers in all other fields, e.g. science, engineering or architecture; agriculture; construction; education or childcare administration; service provider managers; property or personal managers, etc.)            67
Manager of advertising, marketing, promotions, public relations or sales                                                                                                                                                              42
Operations manager  (e.g. managers of administrative services or facilities; computer or information systems; compensation, benefits or purchasing, finance; human resources or training; industrial production; logistics; etc.)     38
Other occupation (please specify):                                  

In [5]:
def get_final_role_category(final_role_type):
    if (final_role_type == " Top executive (e.g. chief executives, general and operations managers of multiple departments or locations, legislators)"):
        category = "Executive"
    elif (final_role_type == "Other manager  (managers in all other fields, e.g. science, engineering or architecture; agriculture; construction; education or childcare administration; service provider managers; property or personal managers, etc.)"):
        category = "Other manager"
    elif (final_role_type == "Manager of advertising, marketing, promotions, public relations or sales "):
        category = "Advertising manager"
    elif (final_role_type == "Operations manager  (e.g. managers of administrative services or facilities; computer or information systems; compensation, benefits or purchasing, finance; human resources or training; industrial production; logistics; etc.)"):
        category = "Operations manager"
    else:
        category = "Other"
    return category

survey_df['job_category'] = survey_df['final_role_type'].apply(get_final_role_category)
survey_df['job_category'].value_counts()

Executive              248
Other manager           67
Other                   47
Advertising manager     42
Operations manager      38
Name: job_category, dtype: int64

## Final wordcloud

In [7]:
from collections import Counter
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stopwords_dict = Counter(stop_words)

In [9]:
titles_df = survey_df[survey_df['Job title Nov/Dec 2022'].notnull()]
print(len(titles_df))
print(titles_df['job_category'].value_counts())

315
Executive              200
Other manager           53
Operations manager      24
Advertising manager     20
Other                   18
Name: job_category, dtype: int64


In [11]:
executive_df = titles_df[titles_df['job_category'] == "Executive"]
print(len(executive_df))
other_manager_df = titles_df[titles_df['job_category'] == "Other manager"]
print(len(other_manager_df))
operations_manager_df = titles_df[titles_df['job_category'] == "Operations manager"]
print(len(operations_manager_df))
advertising_manager_df = titles_df[titles_df['job_category'] == "Advertising manager"]
print(len(advertising_manager_df))
other_df = titles_df[titles_df['job_category'] == "Other"]
print(len(other_df))

200
53
24
20
18


In [12]:
all_text = titles_df['Job title Nov/Dec 2022'].to_list()
all_text = ' '.join(all_text)
print(len(all_text))
all_text = ' '.join([word for word in all_text.split() if word not in stopwords_dict])
print(len(all_text))
print("---------------")

executive_text = executive_df['Job title Nov/Dec 2022'].to_list()
executive_text = ' '.join(executive_text)
executive_text = executive_text.replace("/", " ")
executive_text = executive_text.replace(",", " ")
executive_text = executive_text.replace(".", " ")
executive_text = executive_text.replace("&", " ")
executive_text = executive_text.replace("II", "")
executive_text = executive_text.replace("@", "")
executive_text = executive_text.replace("The", "")
print(len(executive_text))
executive_text_string = ' '.join([word for word in executive_text.split() if word not in stopwords_dict])
executive_text = [word for word in executive_text.split() if word not in stopwords_dict]
print(len(executive_text))
executive_text = list(set(executive_text))
print(len(executive_text))
print("---------------")

other_manager_text = other_manager_df['Job title Nov/Dec 2022'].to_list()
other_manager_text = ' '.join(other_manager_text)
other_manager_text = other_manager_text.replace("/", " ")
other_manager_text = other_manager_text.replace(",", " ")
other_manager_text = other_manager_text.replace(".", " ")
other_manager_text = other_manager_text.replace("&", " ")
other_manager_text = other_manager_text.replace("II", "")
other_manager_text = other_manager_text.replace("@", "")
other_manager_text = other_manager_text.replace("The", "")
print(len(other_manager_text))
other_manager_text_string = ' '.join([word for word in other_manager_text.split() if word not in stopwords_dict])
other_manager_text = [word for word in other_manager_text.split() if word not in stopwords_dict]
print(len(other_manager_text))
other_manager_text = list(set(other_manager_text))
print(len(other_manager_text))
print("---------------")

operations_manager_text = operations_manager_df['Job title Nov/Dec 2022'].to_list()
operations_manager_text = ' '.join(operations_manager_text)
operations_manager_text = operations_manager_text.replace("/", " ")
operations_manager_text = operations_manager_text.replace(",", " ")
operations_manager_text = operations_manager_text.replace(".", " ")
operations_manager_text = operations_manager_text.replace("&", " ")
operations_manager_text = operations_manager_text.replace("II", "")
operations_manager_text = operations_manager_text.replace("@", "")
operations_manager_text = operations_manager_text.replace("The", "")
print(len(operations_manager_text))
operations_manager_text_string = ' '.join([word for word in operations_manager_text.split() if word not in stopwords_dict])
operations_manager_text = [word for word in operations_manager_text.split() if word not in stopwords_dict]
print(len(operations_manager_text))
operations_manager_text = list(set(operations_manager_text))
print(len(operations_manager_text))
print("---------------")


advertising_manager_text = advertising_manager_df['Job title Nov/Dec 2022'].to_list()
advertising_manager_text = ' '.join(advertising_manager_text)
advertising_manager_text = advertising_manager_text.replace("/", " ")
advertising_manager_text = advertising_manager_text.replace(",", " ")
advertising_manager_text = advertising_manager_text.replace(".", " ")
advertising_manager_text = advertising_manager_text.replace("&", " ")
advertising_manager_text = advertising_manager_text.replace("II", "")
advertising_manager_text = advertising_manager_text.replace("@", "")
print(len(advertising_manager_text))
advertising_manager_text_string = ' '.join([word for word in advertising_manager_text.split() if word not in stopwords_dict])
advertising_manager_text = [word for word in advertising_manager_text.split() if word not in stopwords_dict]
print(len(advertising_manager_text))
advertising_manager_text = list(set(advertising_manager_text))
print(len(advertising_manager_text))
print("---------------")


other_text = other_df['Job title Nov/Dec 2022'].to_list()
other_text = ' '.join(other_text)
other_text = other_text.replace("/", " ")
other_text = other_text.replace(",", " ")
other_text = other_text.replace(".", " ")
other_text = other_text.replace("&", " ")
other_text = other_text.replace("II", "")
other_text = other_text.replace("@", "")
other_text = other_text.replace("The", "")
print(len(other_text))
other_text_string = ' '.join([word for word in other_text.split() if word not in stopwords_dict])
other_text = [word for word in other_text.split() if word not in stopwords_dict]
print(len(other_text))
other_text = list(set(other_text))
print(len(other_text))
print("---------------")

7825
7373
---------------
4330
503
156
---------------
1798
190
97
---------------
740
85
47
---------------
579
63
39
---------------
369
41
31
---------------


In [59]:
import matplotlib.pyplot as plt
from collections import Counter

# Create a frequency distribution
word_frequencies = Counter(all_text.split())
executive_word_frequencies = Counter(executive_text_string.split())
other_manager_word_frequencies = Counter(other_manager_text_string.split())
advertising_manager_word_frequencies = Counter(advertising_manager_text_string.split())
operations_manager_word_frequencies = Counter(operations_manager_text_string.split())
other_word_frequencies = Counter(other_text_string.split())

# Generate a scatter plot with text at custom positions
plt.figure(figsize=(10, 5), dpi=100)
    
for number, word in enumerate(other_text):
    # Create random coordinates for each word within a cluster:
    x = np.random.normal(0, 1.5, len(other_text))
    y = np.random.normal(1, 1.5, len(other_text))
    #Set the font size based on the frequency of the word:
    fontsize = other_word_frequencies.get(word, 1) * 15
    plt.text(x[number], y[number], word, fontsize=fontsize, color='mediumorchid', ha='center', va='center')
    
for number, word in enumerate(executive_text):
    # Create random coordinates for each word within a cluster
    x = np.random.normal(2.5, 2.5, len(executive_text))
    y = np.random.normal(2.5, 2.5, len(executive_text))
    #Set the font size based on the frequency of the word
    fontsize = executive_word_frequencies.get(word, 1) * 15  
    plt.text(x[number], y[number], word, fontsize=fontsize, color = 'royalblue', ha='center', va='center')
    
for number, word in enumerate(other_manager_text):
    # Create random coordinates for each word within a cluster
    x = np.random.normal(7, 1.5, len(other_manager_text))
    y = np.random.normal(0, 1.5, len(other_manager_text))
    #Set the font size based on the frequency of the word
    fontsize = other_manager_word_frequencies.get(word, 1) * 15  
    plt.text(x[number], y[number], word, fontsize=fontsize, color='orange', ha='center', va='center')
    
for number, word in enumerate(advertising_manager_text):
    # Create random coordinates for each word within a cluster
    x = np.random.normal(7, 1.5, len(advertising_manager_text))
    y = np.random.normal(7, 1.5, len(advertising_manager_text))
    #Set the font size based on the frequency of the word
    fontsize = advertising_manager_word_frequencies.get(word, 1) * 15  # Adjust the multiplier as needed
    plt.text(x[number], y[number], word, fontsize=fontsize, color='tomato', ha='center', va='center')
    
for number, word in enumerate(operations_manager_text):
    # Create random coordinates for each word within a cluster:
    x = np.random.normal(0, 1.5, len(operations_manager_text))
    y = np.random.normal(7, 1.5, len(operations_manager_text))
    #Set the font size based on the frequency of the word:
    fontsize = operations_manager_word_frequencies.get(word, 1) * 15  #Adjust the multiplier as needed
    plt.text(x[number], y[number], word, fontsize=fontsize, color='limegreen', ha='center', va='center')

# Display the scatter plot
plt.axis("off")
plt.show()